In [2]:
from pathlib import Path

from sqlalchemy import create_engine, String, Integer, Column, Unicode, ForeignKey, URL, select, and_
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, declarative_base, Session, relationship
import pandas as pd

In [3]:
# задание базы данных
engine = create_engine("sqlite:///mosmetro.db", echo=False)

In [5]:
Base = declarative_base() # задание родительского класса, чтобы описывать каждую таблицу как класс

# создание таблиц
class Line(Base):
    __tablename__ = 'line'

    id = Column(Integer, primary_key=True)
    line = Column(Unicode)
    station = relationship('Station', uselist=False, back_populates='line')

    def __repr__(self) -> str:
        return f"Line name: {self.line}"


class District(Base):
    __tablename__ = 'district'

    id = Column(Integer, primary_key=True, autoincrement='auto', index=True)
    district = Column(Unicode)
    station = relationship('Station', uselist=False, back_populates='district')

    def __repr__(self) -> str:
        return f"District name: {self.district}"


class MCD(Base):
    __tablename__ = 'mcd'

    id = Column(Integer, primary_key=True, autoincrement='auto', index=True)
    name = Column(Unicode)
    global_id = Column(Integer)
    station = relationship('Station', uselist=False, back_populates='mcd')

    def __repr__(self) -> str:
        return f"MCD station name: {self.name}"


class AeroExpress(Base):
    __tablename__ = 'aeroexpress'

    id = Column(Integer, primary_key=True, autoincrement='auto', index=True)
    name = Column(Unicode)
    global_id = Column(Integer)
    station = relationship('Station', uselist=False, back_populates='aeroexpress')

    def __repr__(self) -> str:
        return f"Aeroexpress station name: {self.name}"


class RailwayStation(Base):
    __tablename__ = 'railway'

    id = Column(Integer, primary_key=True, autoincrement='auto', index=True)
    name = Column(Unicode)
    global_id = Column(Integer)
    station = relationship('Station', uselist=False, back_populates='railway')

    def __repr__(self) -> str:
        return f"Railway station name: {self.name}"


class Station(Base):
    __tablename__ = 'station'

    id = Column(Integer, primary_key=True, autoincrement='auto', index=True)
    dataset_id = Column(Integer)
    station = Column(Unicode)
    line_id = mapped_column(ForeignKey("line.id", ondelete='CASCADE'), nullable=False)
    line = relationship("Line", back_populates="station")
    adm_area = Column(Unicode)
    district_id = mapped_column(ForeignKey("district.id", ondelete='CASCADE'), nullable=False)
    district = relationship("District", back_populates="station")
    mcd_id = mapped_column(ForeignKey("mcd.id", ondelete='CASCADE'))
    mcd = relationship("MCD", back_populates="station")
    aeroexpress_id = mapped_column(ForeignKey("aeroexpress.id", ondelete='CASCADE'))
    aeroexpress = relationship("AeroExpress", back_populates="station")
    railway_id = mapped_column(ForeignKey("railway.id", ondelete='CASCADE'))
    railway = relationship("RailwayStation", back_populates="station")
    status = Column(Unicode)
    global_id = Column(Integer)

    def __repr__(self) -> str:
        return f"Station name: {self.station}"

In [4]:
"""Функция для создания бд"""
def json_to_db_metro(path_to_json: str) -> None:
    Base.metadata.create_all(engine) # создание таблиц на основе классов от родительского класса Base
    data = pd.read_json(Path(path_to_json), orient='records', encoding='utf-8')
    with Session(engine) as session:
        for row in data['Line'].unique():
            line = Line(
                line=row
            )
            session.add(line)
            session.commit()
    with Session(engine) as session:
        for row in data['District'].unique():
            district = District(
                district=row
            )
            session.add(district)
            session.commit()
    with Session(engine) as session:
        for i, row in data.loc[data['MCDStation'].str.len() != 0].iterrows():
            mcd = MCD(
                name=row["MCDStation"]['value'],
                global_id=row["MCDStation"]['global_id']
            )
            session.add(mcd)
            session.commit()
    with Session(engine) as session:
        for i, row in data.loc[data['AeroexpressStation'].str.len() != 0].iterrows():
            aeroexpress = AeroExpress(
                name=row["AeroexpressStation"]['value'],
                global_id=row["AeroexpressStation"]['global_id']
            )
            session.add(aeroexpress)
            session.commit()
    with Session(engine) as session:
        for i, row in data.loc[data['RailwayStation'].str.len() != 0].iterrows():
            railway = RailwayStation(
                name=row["RailwayStation"]['value'],
                global_id=row["RailwayStation"]['global_id']
            )
            session.add(railway)
            session.commit()
    with Session(engine) as session:
        for i, row in data.iterrows():
            line_id = select(Line.id).where(Line.line == row['Line'])
            print(line_id)
            district_id = select(District.id).where(District.district == row['District'])
            if isinstance(row['MCDStation'], dict):
                mcd_id = select(MCD.id).where(MCD.name == row['MCDStation']['value'])
            else:
                mcd_id = None
            if isinstance(row['AeroexpressStation'], dict):
                aeroexpress_id = select(AeroExpress.id).where(AeroExpress.name == row['AeroexpressStation']['value'])
            else:
                aeroexpress_id = None
            if isinstance(row['RailwayStation'], dict):
                railway_id = select(RailwayStation.id).where(RailwayStation.name == row['RailwayStation']['value'])
            else:
                railway_id = None
            station = Station(
                dataset_id=row['ID'],
                station=row['Station'],
                line_id=line_id,
                adm_area=row['AdmArea'],
                district_id=district_id,
                mcd_id=mcd_id,
                aeroexpress_id=aeroexpress_id,
                railway_id=railway_id,
                status=row['ObjectStatus'],
                global_id=row['global_id']
            )
            session.add(station)
            session.flush() # обновление связей
            session.commit()


In [5]:
json_to_db_metro('mosmetro.json')

2023-06-16 00:32:32,262 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-16 00:32:32,263 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("line")
2023-06-16 00:32:32,264 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-16 00:32:32,266 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("line")
2023-06-16 00:32:32,267 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-16 00:32:32,268 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("district")
2023-06-16 00:32:32,270 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-16 00:32:32,271 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("district")
2023-06-16 00:32:32,272 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-16 00:32:32,273 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("mcd")
2023-06-16 00:32:32,274 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-16 00:32:32,276 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("mcd")
2023-06-16 00:32:32,277 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-16 

C:\Users\HP\AppData\Local\Temp\ipykernel_4412\3688753687.py:72: SAWarning: implicitly coercing SELECT object to scalar subquery; please use the .scalar_subquery() method to produce a scalar subquery.
  session.flush()


SELECT line.id 
FROM line 
WHERE line.line = :line_1
2023-06-16 00:32:35,875 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-16 00:32:35,877 INFO sqlalchemy.engine.Engine INSERT INTO station (dataset_id, station, line_id, adm_area, district_id, mcd_id, aeroexpress_id, railway_id, status, global_id) VALUES (?, ?, (SELECT line.id 
FROM line 
WHERE line.line = ?), ?, (SELECT district.id 
FROM district 
WHERE district.district = ?), ?, ?, ?, ?, ?)
2023-06-16 00:32:35,877 INFO sqlalchemy.engine.Engine [cached since 0.1822s ago] (90, 'ВДНХ', 'Калужско-Рижская линия', 'Северо-Восточный административный округ', 'Останкинский район', None, None, None, 'действует', 58701973)
2023-06-16 00:32:35,879 INFO sqlalchemy.engine.Engine COMMIT
SELECT line.id 
FROM line 
WHERE line.line = :line_1
2023-06-16 00:32:35,886 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-16 00:32:35,888 INFO sqlalchemy.engine.Engine INSERT INTO station (dataset_id, station, line_id, adm_area, district_id, mcd_id

In [6]:
with Session(engine) as session:
    # Все станции, id в датасете которых больше ста и которые располагаются на солнцевской линии
    print(session.query(Station).filter(and_(Station.dataset_id > 100, Station.line_id == select(Line.id).where(Line.line == 'Солнцевская линия').scalar_subquery())).all())

    # Все станции, у которых есть выход на мцд, аэроэкспресс и ж\д станцию
    print(session.query(Station).filter(and_(Station.mcd_id != None, Station.aeroexpress_id != None, Station.railway_id != None)).all())

    # Все действующие станции в районе Арбат на Арбатско-Покровской линии в ЦАО
    print(session.query(Station).filter(and_(Station.status == 'действует', Station.district_id == select(District.id).where(District.district == 'район Арбат').scalar_subquery(), Station.line_id == select(Line.id).where(Line.line == 'Арбатско-Покровская линия').scalar_subquery(), Station.adm_area == 'Центральный административный округ')).all())

     # Все действующие станции в ЦАО с ж\д станциями
    print(session.query(Station).filter(and_(Station.status == 'действует', Station.adm_area == 'Центральный административный округ', Station.railway_id != None)).all())

     # Все станции МЦД, Аэроэкспресса и Ж/Д, когда они все на одной станции метро
    print(session.query(Station, MCD, AeroExpress, RailwayStation).filter(and_(Station.mcd_id == MCD.id, Station.aeroexpress_id == AeroExpress.id, Station.railway_id == RailwayStation.id)).all())



[Station name: Плющиха, Station name: Новопеределкино, Station name: Дорогомиловская, Station name: Третьяковская, Station name: Парк Победы, Station name: Ломоносовский проспект, Station name: Боровское шоссе, Station name: Рассказовка, Station name: Мичуринский проспект, Station name: Деловой центр, Station name: Минская, Station name: Солнцево, Station name: Волхонка, Station name: Раменки, Station name: Озёрная, Station name: Говорово]
[Station name: Белорусская, Station name: Фили, Station name: Кунцевская, Station name: Международная, Station name: Беговая, Station name: Савёловская, Station name: Деловой центр, Station name: Белорусская, Station name: Окружная, Station name: Кунцевская, Station name: Деловой центр, Station name: Славянский бульвар, Station name: Кунцевская, Station name: Выставочная, Station name: Шелепиха, Station name: Деловой центр, Station name: Окружная, Station name: Шелепиха, Station name: Савёловская]
[Station name: Арбатская, Station name: Смоленская]
[